### **Instalação de Principais Módulos**

In [ ]:
!pip install tensorflow tensorflow-gpu opencv-python matplotlib


!pip3 install --upgrade tensorflow-gpu


!pip install "tensorflow-gpu"

from IPython.display import clear_output
clear_output(wait=False)

### **Importação de Bibliotecas**

In [ ]:
import tensorflow as tf
import os

import cv2
import imghdr

import numpy as np
from matplotlib import pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

### **Indicar caminho do banco de Imagens**

In [ ]:
data_dir = 'data'

*   **Quantidade de Classes**

In [ ]:
data = tf.keras.utils.image_dataset_from_directory('data')

### **Leitura de Imagens e Classificação Primaria**

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

*   **Amostragem de Pixels**

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
data.as_numpy_iterator().next()

### **Definição de Métricas para Treinamento**

*   **Separação de amostras**

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)

*   **Quantidade de treinamento**

In [ ]:
train_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
train

### **Criação de Modelo**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

*   **Saída de resultados de treinamento do modelo (Epochs)**

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

*   **Treinamento**

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

*   **Valores de Precisão e Perda**

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='Perda')
plt.plot(hist.history['val_loss'], color='orange', label='Perda_Validação')
fig.suptitle('Perda', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='Precisão')
plt.plot(hist.history['val_accuracy'], color='orange', label='Precisão_Validação')
fig.suptitle('Precisão', fontsize=20)
plt.legend(loc="upper left")
plt.show()

### **Evolução do Modelo**

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

*   **Aproximar Valores de avaliação**

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(pre.result(), re.result(), acc.result())

### **Analise de imagem e recorte dos olhos**

In [ ]:
import cv2 
img = cv2.imread('PedroRGB2.jpg') 
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
olhos_cascade = cv2.CascadeClassifier('haarcascade_righteye_2splits.xml')
olhos = olhos_cascade.detectMultiScale(gray, 1.1, 4) 
for (x, y, w, h) in olhos: 
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), 2) 
    olhos = img[y:y + h, x:x + w] 
    cv2.imshow("olho",olhos)
    cinza = cv2.cvtColor(olhos, cv2.COLOR_BGR2GRAY)
    cinza2 = cv2.imwrite('olhos.jpeg', cinza)       

cv2.imwrite('detcted.jpeg', img)
cv2.imshow('img', img) 
cv2.waitKey()

*   **Imagem final**

In [ ]:
olho = cv2.imread('olhos.jpeg') 
resize = tf.image.resize(olho, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

### **Predição do modelo com "Y-hat"**

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.9: 
    print(f'O olho está ABERTO')
else:
    print(f'O olho está FECHADO')